In [20]:
# A partir del conjunto de datos creado en el ejercicio 7.1.
# a) Ajusta el modelo de regresión logística múltiple con todas las variables del dataset excepto 'ocean_proximity', 'longitude' y 'latitude', como
# predictoras de la variable 'oceano'.
# b) ¿Qué probabilidad tendría una casa con las siguientes características de ser oceano=1?
# housing_median_age = 35 
# total_rooms = 5000
# total_bedrooms = 1200
# population = 100000
# households = 1000
# median_income = 10 (decenas de miles de dólares)
# median_house_value = 250000

In [22]:
# Apartado a)
# Se carga el dataset y se crea la nueva variable 'oceano'
import pandas as pd

housing = pd.read_csv('datasets/housing/housing.csv')

# Se crea la variable categórica binomial 'oceano'. Se añade como otra columna del dataframe.
housing['oceano'] = [1 if x=='<1H OCEAN' or x=='NEAR OCEAN' else 0 for x in housing['ocean_proximity']]

# Se eliminan del dataset las variables 'ocean_proximity', 'longitude' y 'latitude'
housing_num = housing.drop('ocean_proximity', axis=1)
housing_num = housing_num.drop('longitude', axis=1)
housing_num = housing_num.drop('latitude', axis=1)

# Se elimina la variable respuesta para que en el dataset solo queden las variables predictoras
housing_num = housing_num.drop('oceano', axis=1)

In [24]:
# Se lleva a cabo imputación (faltan datos de 'total_bedrooms') + estandarización
from sklearn.impute import SimpleImputer # Para imputación
from sklearn.preprocessing import StandardScaler # Para estandarización
from sklearn.pipeline import Pipeline # Para combinar estandarización e imputación
from sklearn.compose import ColumnTransformer # Para considerar la variable categórica en la imputación y estandarización
from sklearn.linear_model import LogisticRegression # Para regresión logística

# Se define el pipeline
num_pipeline = Pipeline([
    ('imputador', SimpleImputer(strategy='median')),
    ('std_scaler', StandardScaler()),
])

# Se aplica el pipeline
housing_num_tr = num_pipeline.fit_transform(housing_num)

# Nombre variables numéricas predictoras
num_attribs = list(housing_num)

# Se define el full pipeline
full_pipeline = ColumnTransformer([
    ('num', num_pipeline, num_attribs)
])

# Se aplica el full pipeline
housing_prepared = full_pipeline.fit_transform(housing_num)

# Variables predictoras
x = housing_prepared

# Variable respuesta
y = housing['oceano'].values.reshape(-1,1).ravel()

# Ajuste del modelo de regresión logística
logistic_reg = LogisticRegression()
logistic_reg.fit(x,y)

print('El intercepto es:', logistic_reg.intercept_)
print('Los coeficientes beta son:', logistic_reg.coef_)

El intercepto es: [0.40566911]
Los coeficientes beta son: [[-0.07383131 -2.00799946  0.12206637  1.36485265  0.52288925  0.04659429
   1.17541298]]


In [26]:
# Se pueden presentar los resultados en forma de tabla
from tabulate import tabulate

coefs = logistic_reg.coef_.tolist()[0]
tabla = zip(num_attribs,coefs)
print(tabulate(tabla,headers=['Variable','Coeficiente']))

Variable              Coeficiente
------------------  -------------
housing_median_age     -0.0738313
total_rooms            -2.008
total_bedrooms          0.122066
population              1.36485
households              0.522889
median_income           0.0465943
median_house_value      1.17541


In [28]:
# Apartado b)
# Nueva observación
import numpy as np

housing_new = pd.DataFrame(np.array([[35, 2000, 1000, 1200, 900, 10, 250000]]), columns=num_attribs)

# Estandarización
scaler = StandardScaler()
scaler.fit(housing_num)
housing_new_prepared = scaler.transform(housing_new)

# Predicción del modelo
clase_predicha = logistic_reg.predict(housing_new_prepared)
print(clase_predicha)

[1]


In [30]:
# La probabilidad proporcionada por el modelo es:
print(logistic_reg.predict_proba(housing_new_prepared))

[[0.12420925 0.87579075]]


In [45]:
# Al considerar más variables predictoras además de 'median_house_value', como se hizo en el ejercicio 7.1., las probabilidades de que oceano=1 para 
# la nueva observación aumentan

In [32]:
housing_new_prepared

array([[ 0.50539419, -0.29142558,  1.09671839, -0.19910795,  1.04744666,
         3.22634352,  0.37388967]])